# Characterisic values

In this notebook you will learn the meaning of various characteristic values, and how to calculate them.

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa import seasonal
from datetime import datetime

from datascience.read import Era5, AscatDataH121, read_multiple_ds

%matplotlib widget

We will look at data from ASCAT and ERA5

In [ ]:
era5 = Era5(read_bulk = False)
ascat = AscatDataH121(read_bulk = False)

In [ ]:
lat = 48.198905
lon = 16.367182
gpi = era5.grid.find_nearest_gpi(lon, lat)[0]

In [ ]:
ts = read_multiple_ds(loc=(lon, lat), ascat=ascat, era5=era5, ref_ds= "ascat")

You can get the needed variables, and calculate indices with them:

In [ ]:
t2m = ts["t2m_era5"]
tp = ts["tp_era5"]
ssm = ts["surface_soil_moisture"]

TCI is used to determine stress on vegetation caused by temperatures and excessive wetness. Conditions are estimated relative to the maximum and minimum temperatures and modified to reflect different vegetation responses to temperature. TCI varies from 0, for extremely unfavorable conditions to 1 for optimal conditions.

In [ ]:
tci = (t2m.max()-t2m)/(t2m.max()-t2m.min())

PCI ranges from 0 to 1 corresponding to changes in precipitation from extremely unfavorable to optimal. In case of a meteorological drought which has an extremely low precipitation, the PCI is close or equal to 0, and at flooding conditions, the PCI is close to 1. 

In [ ]:
pci = (tp-tp.min())/(tp.max()-tp.min())

SCI ranges from 0 to 1 corresponding to changes in soil moisture from extremely unfavorable to optimal. In case of a meteorological drought which has an extremely low soil moisture, the SCI is close or equal to 0, and at moisture saturation conditions, the SCI is close to 1.

In [ ]:
sci = (ssm-ssm.min())/(ssm.max()-ssm.min())

You can also compare these indices visually: 

In [ ]:
sci_resampled = sci.resample('5D').mean()
pci_resampled = pci.resample('5D').mean()
tci_resampled = tci.resample('5D').mean()

trend_sci = seasonal.seasonal_decompose(sci_resampled.dropna(), model="additive", period=73).trend
trend_pci = seasonal.seasonal_decompose(pci_resampled.dropna(), model="additive", period=73).trend
trend_tci = seasonal.seasonal_decompose(tci_resampled.dropna(), model="additive", period=73).trend

fig, ax = plt.subplots(1,1, figsize=(10,5))
ax.plot(trend_sci, label="sci")
ax.plot(trend_tci, label="tci")
ax1 = ax.twinx()
ax1.plot(trend_pci, c="r", label="pci")
ax.set_title("SCI, PCI and TCI trends")
ax.set_ylabel("SCI/TCI trend")
ax1.set_ylabel("PCI trend")
ax.set_xlabel("Date")
ax.legend()
ax1.legend()

plt.show()

## SMADI/pytesmo

A package for calculating and analysing time-series Data is the SMADI library (https://github.com/MuhammedM294/smadi/tree/main)

In [ ]:
from smadi import preprocess

You can calculate the mean daily value of a variable with the *compute_clim* function. But also other statistical values for various timeframes.

In [ ]:
ts_smadi = preprocess.compute_clim(ts, "day", "surface_soil_moisture", ["mean"])
ts_smadi

If you substract the soil moisture value from the daily mean you get deviations:

In [ ]:
ts_smadi["dev"] = ts_smadi["surface_soil_moisture"]-ts_smadi["norm-mean"]

You can also show the deviations visually:

In [ ]:
kwargs = {"facecolors": "None", "edgecolor": "C0"}

fig, ax = plt.subplots(figsize=(13,5))
ax.plot(ts_smadi["norm-mean"], c="r")
ax.scatter(ts_smadi.index, ts_smadi["surface_soil_moisture"], marker="o", **kwargs)
ax.set_title("Deviation of surface soil moisture compared to daily mean")
ax.set_xlabel("Date")
ax.set_ylabel("surface soil moisture [%]")
ax.set_xlim(datetime(2008,1,1), datetime(2012,1,1))

plt.show()

Alternatively you can use pytesmo (https://github.com/TUW-GEO/pytesmo/tree/master) to plot and calculate anomalies:

In [ ]:
from pytesmo.time_series import plotting

In [ ]:
ts_pytesmo = ts[["surface_soil_moisture", "swvl1_era5", "stl1_era5"]]

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(13,10))
plotting.plot_clim_anom(ts_pytesmo, axes=axes)
for ax in axes:
    ax.set_xlim(datetime(2008,1,1), datetime(2012,1,1))

plt.tight_layout()
plt.show()